In [52]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np

In [53]:
genre_dict = {
    'Action': '1',
    'Adventure':'2',
    'Cars':'3',
    'Comedy':'4',
    'Dementia':'5',
    'Demons':'6',
    'Drama':'8',
    'Ecchi':'9',
    'Fantasy':'10',
    'Game':'11',
    'Historical':'13',
    'Horror':'14',
    'Kids':'15',
    'Harem':'35',
    'Josei':'43'
}

In [74]:
def convert_to_number(num):
    result = 0
    num_conversion = {'K':1000, 'M':1000000, 'B':1000000000}
    if num.isdigit():
        result = int(num)
    else:
        if len(num) > 1:
            result = float(num[:-1]) * num_conversion.get(num[-1].upper(), 1)
    return int(result)

def get_genre_webpage(genre):
    site_url = 'https://myanimelist.net/anime/genre/'
    final_url = site_url + genre_dict[genre]+ '/' + genre
    response = requests.get(final_url)
    if response.status_code != 200:
        print('Status code:', response.status_code)
        raise Exception('Failed to fetch web page ' + final_url)
    return BeautifulSoup(response.text) 

def create_lists(titles, producers, episodes, ratings, members, infos):
    titles_list = []; producers_list = []; episodes_list = []; infos_list = []; ratings_list = []; members_list =[]

    for title in titles:
        name = title.find('a').text
        titles_list.append(name)
    for producer in producers:
        studios_list = []
        tags = producer.find('div', class_='property').find_all('span', class_='item')
        for tag in tags:
            studio = tag.find('a').text
            studios_list.append(studio)
        producers_list.append(studios_list)
    for episode in episodes:
        e = episode.find_all('span', class_='item', limit=3)
        e = e[2].find_all('span')
        e = e[0].text.split('eps')
        try:
            e = int(e[0])
        except ValueError:
            e = 0
        episodes_list.append(e)
    for info in infos:
        i = info.find('a')['href']
        infos_list.append(i)
    for rating in ratings:
        r = rating.get_text().strip().split('\n')
        m = r[0]
        ratings_list.append(m)
    for member in members:
        m = member.get_text().strip().split('\n')
        m = m[3]
        m = convert_to_number(m)
        members_list.append(m)

    return titles_list, producers_list, episodes_list, infos_list, ratings_list, members_list


def create_df(genre):
     
    doc = get_genre_webpage(genre)
    
    titles = doc.find_all('h2', class_ = 'h2_anime_title')
    infos = doc.find_all('h2', class_ = 'h2_anime_title')
    producers = doc.find_all('div', class_ ='properties')
    episodes = doc.find_all('div', class_='prodsrc')
    ratings = doc.find_all('div', class_= 'scormem')
    members = doc.find_all('div', class_= 'scormem')
    
   
    if len(titles) == len(producers) == len(episodes) == len(infos) == len(ratings) == len(members):
        titles_list, producers_list, episodes_list, infos_list , ratings_list , members_list = create_lists(titles, producers, episodes, ratings, members, infos)
        
        column_names = ['Title', 'Producer', 'Episodes', 'Rating', 'Members','Anime Information Url']
        df = pd.DataFrame(list(zip(titles_list, producers_list, episodes_list, ratings_list, members_list, infos_list)), columns = column_names)
        
        
    else: print("Length of lists are not equal, dataframe cannot be created")
     
    
    return df

In [75]:
create_df('Adventure')

,Title,Producer,Episodes,Rating,Members,Anime Information Url
0,Fullmetal Alchemist: Brotherhood,[Bones],64,9.15,2800000,https://myanimelist.net/anime/5114/Fullmetal_A...
1,Sword Art Online,[A-1 Pictures],25,7.20,2700000,https://myanimelist.net/anime/11757/Sword_Art_...
2,Naruto,[Studio Pierrot],220,7.96,2400000,https://myanimelist.net/anime/20/Naruto
3,Hunter x Hunter (2011),[Madhouse],148,9.05,2300000,https://myanimelist.net/anime/11061/Hunter_x_H...
4,No Game No Life,[Madhouse],12,8.12,2100000,https://myanimelist.net/anime/19815/No_Game_No...
...,...,...,...,...,...,...
95,Sora yori mo Tooi Basho,[Madhouse],13,8.54,424000,https://myanimelist.net/anime/35839/Sora_yori_...
96,Maoyuu Maou Yuusha,[Arms],12,7.26,423000,https://myanimelist.net/anime/14833/Maoyuu_Mao...
97,Magi: Sinbad no Bouken (TV),[Lay-duce],13,7.86,422000,https://myanimelist.net/anime/31741/Magi__Sinb...
98,Fairy Tail: Final Series,"[A-1 Pictures, Bridge, CloverWorks]",51,7.55,419000,https://myanimelist.net/anime/35972/Fairy_Tail...
